## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,Sentence #,Word,POS,Tag
20,1.0,from,IN,O
21,1.0,that,DT,O
22,1.0,country,NN,O
23,2.0,families,NNS,O
24,2.0,of,IN,O
25,2.0,soldier,NNS,O
26,2.0,kill,VBN,O
27,2.0,in,IN,O
28,2.0,the,DT,O
29,2.0,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'this', 'week', 'restart', 'part', 'of'],
 Class: geo Features: ['the', 'conversion', 'process', 'at', 'its', 'nuclear', 'plant', '<E5>', '<E4>', '<E3>'],
 Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'official', 'say', 'they', 'expect', 'to'],
 Class: tim Features: ['sensitive', 'part', 'of', 'the', 'plant', 'after', 'an', 'surveillance', 'system', 'begin'],
 Class: org Features: ['of', 'the', 'plant', 'after', 'an', 'surveillance', 'system', 'begin', 'function', '<E5>'],
 Class: org Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'the', 'with', 'backing', 'have', 'threaten', 'to'],
 Class: gpe Features: ['backing', 'have', 'threaten', 'to', 'refer', 'to', 'the', 'which', 'could', 'impose'],
 Class: gpe Features: ['impose', 'sanction', 'if', 'it', 'find', 'have', 'violate', 'the', 'treaty', '<E5>'],
 Class: art Features: ['it', 'find', 'have', 'violate', 'the', 'treaty', '<E5>', '<E4>', '<E3>', '<E2>'],
 Class:

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,class_label,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,1.291311,-1.395655,-0.842288,-0.204513,-0.320744,0.825330,0.324720,0.305276,-0.655858,...,-0.028828,0.040447,-0.022867,0.007758,-0.001432,0.050254,-0.033366,-0.030433,-0.024755,-0.055527
21,geo,1.252882,1.062832,-0.100958,-0.661689,-0.059914,-0.142298,-0.116525,-0.095413,0.229158,...,0.015232,0.036491,-0.009808,-0.011257,-0.009423,0.027827,0.014583,-0.036247,0.021659,-0.030257
22,gpe,1.396641,-1.627629,-0.947095,-0.086366,0.448235,0.122432,-0.518810,-0.089908,-0.417823,...,-0.001076,-0.043413,0.004238,-0.006224,-0.003356,0.049970,0.009859,0.006272,0.014570,-0.022867
23,tim,0.794307,0.120203,0.876269,-0.324209,-0.208115,0.601217,0.209155,0.243444,-0.239391,...,0.028134,-0.003874,0.049644,-0.012035,0.029009,0.029719,-0.072605,-0.017597,-0.075692,-0.035778
24,org,1.035625,0.475962,0.630110,-0.408557,-0.177670,0.581814,0.216845,0.226088,-0.209471,...,0.031337,-0.026504,0.050084,-0.005293,0.050210,0.004857,-0.052155,0.002646,-0.058958,-0.015115
25,org,1.802081,-1.336901,-0.148628,-0.392049,0.502681,0.002994,-0.671331,-0.348951,-0.499199,...,-0.007009,-0.003595,0.004889,-0.044761,-0.020777,0.013063,0.022696,0.019419,0.007421,-0.028972
26,gpe,0.951219,-0.021211,0.799856,-0.136242,1.401605,0.096429,-1.159761,-0.405659,-0.670479,...,0.018888,0.091334,-0.016006,-0.018877,-0.036288,0.034650,0.031553,0.047860,0.013174,-0.016222
27,gpe,0.920474,0.294226,0.403068,-0.413949,0.032640,-0.141943,-0.128251,-0.171172,0.103726,...,0.057286,0.051908,-0.166296,0.078878,-0.051745,0.082832,0.063136,-0.020598,0.024595,-0.067378
28,art,1.497734,1.248735,-0.365612,-0.739886,0.007540,-0.157804,-0.172665,-0.145901,0.224588,...,0.011219,0.023124,-0.044994,0.039841,-0.025543,0.032277,-0.003386,0.012816,0.007328,0.010251
29,gpe,1.267708,-1.640130,-0.998705,-0.180379,-0.260564,0.024827,-0.030219,0.020430,-0.037794,...,0.001542,0.024303,0.043314,0.031287,-0.024966,0.094510,-0.018680,-0.013891,0.125081,-0.009433


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 1.64209376e+00, -5.03240934e-02, -1.36572016e+00, ...,
         -4.41816463e-03, -2.87324694e-03,  9.20767793e-03],
        [ 1.27552290e+00, -1.51929097e+00, -9.89196847e-01, ...,
         -6.38495432e-03, -2.33466069e-03,  9.85476569e-03],
        [ 3.62621698e-01,  1.04465035e-01,  1.28206254e-01, ...,
         -1.46990334e-02, -4.20777171e-04,  1.46646805e-01],
        ...,
        [ 1.73496809e+00,  2.50131200e-01,  1.64837100e+00, ...,
         -5.17855211e-02, -1.52513990e-02,  6.36453004e-02],
        [ 1.26953522e+00,  1.52023079e-01,  8.61445730e-01, ...,
          9.18349493e-02,  4.95490019e-02, -4.49497833e-02],
        [ 1.03963213e+00, -4.61278242e-01,  5.20303269e-01, ...,
          5.91326502e-02, -1.81634542e-01,  1.43552342e-01]]),
 3954    per
 1606    geo
 2490    geo
 2786    geo
 854     gpe
        ... 
 6227    tim
 3680    tim
 4580    tim
 4348    geo
 2384    org
 Name: class_label, Length: 5022, dtype: object,
 array([[ 2.29833242e-01,  6.08954413

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.20006371455877667

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.20006371455877667

In [10]:
test_y.iloc[0]

'per'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [12]:
train_predictions

array(['tim', 'per', 'geo', ..., 'geo', 'geo', 'org'], dtype=object)

In [13]:
train_y

3954    per
1606    geo
2490    geo
2786    geo
854     gpe
       ... 
6227    tim
3680    tim
4580    tim
4348    geo
2384    org
Name: class_label, Length: 5022, dtype: object

In [14]:
test_predictions

array(['gpe', 'org', 'org', ..., 'geo', 'geo', 'tim'], dtype=object)

In [15]:
test_y

3       per
5       geo
7       gpe
16      org
17      geo
       ... 
6268    gpe
6270    gpe
6275    geo
6276    per
6277    tim
Name: class_label, Length: 1256, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

,eve,nat,geo,per,org,tim,gpe,art
eve,0,0,8,1,0,4,1,0
nat,0,2,2,0,0,0,0,0
geo,2,0,211,36,36,44,36,4
per,1,0,45,62,21,27,24,2
org,0,0,57,27,81,31,41,2
tim,0,3,92,14,23,72,18,1
gpe,0,0,54,38,29,11,83,1
art,0,0,2,1,2,2,0,2


In [17]:
a2.confusion_matrix(train_y, train_predictions)

,eve,nat,geo,per,org,tim,gpe,art
eve,12,0,13,0,1,4,0,0
nat,0,10,3,0,0,1,1,0
geo,1,0,1105,77,99,129,128,2
per,0,0,157,313,71,62,97,1
org,0,0,232,91,313,86,107,0
tim,0,0,284,52,75,421,86,0
gpe,0,0,221,94,97,85,450,1
art,0,0,6,2,4,8,1,19


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')